In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
plt.rcParams.update({'font.size': 8, 'pdf.fonttype': 42})

In [3]:
#sns.set_style("whitegrid")
sns.set_style("ticks")

In [39]:
df = pd.read_csv('results/fmri_profile_corr.csv')


In [40]:
df_nc = pd.read_csv('profile_noise_ceiling_single_trial.csv')
df_nc_thd = pd.read_csv('profile_nc_threshold_single_trial.csv')

In [41]:
df_nc['identifier'] = df_nc['Subject'].values + df_nc['ROI'].values + df_nc['Vox_idx'].values.astype(str)
df_nc_thd['identifier'] = df_nc_thd['Subject'].values + df_nc_thd['ROI'].values + df_nc_thd['Vox_idx'].values.astype(str)

df_nc = df_nc.join(df_nc_thd.set_index('identifier'), rsuffix='_', on='identifier')

In [42]:
df['identifier'] = df['Target'].values + df['Target ROI'].values + df['Vox_idx'].values.astype(int).astype(str)

In [43]:
df = df.merge(df_nc, on='identifier', suffixes=('', '_nc'))

In [45]:
# Remove those voxels whose noise ceiling is below the threshold.
df.drop(df.loc[df['Correlation_nc'] <= df['Threshold 1']].index, inplace=True)

In [46]:
# Because of the noise, the profile correlation is still possible to exceed noise ceiling for some voxels. We clip the normalized correlation betwwen -1, 1.
df['Normalized correlation'] = np.clip(np.nan_to_num(df['Correlation'].values / df['Correlation_nc'].values), a_min=-1, a_max=1)